In this notebook, we intend to pretraing the Higgingface Gemma model. We will use 2 billion parameters instruction trained version. Before we beging, we need to request access to the gemma model from huggingface as it is a gated model. To do so, follow the link https://huggingface.co/google/gemma-7b and submit the consent form in Terms. Once you get access to the model, go to the huggingface profile -> settings -> Access Tokens and generate a new token, preferably in READ form. In your google colab, click on the Sectets (key submol) tab in the left sidebar and add your token and use "HF_TOKEN" as name. In google colab, go to connect -> change runtime type -> Connect to T4 GPU.

We will use parameter efficient finetuning technique (PEFT) LORA to finetune the gemma model. Generally, finetuning a LLM with 2 Billion parameters is complex and resource as well as time consuming. LoRA or low rank adaptation enables us to train only a fraction of the weights in the LLM while freezing the other weights. LoRA leverages the rank property of a matrix. Lower rank matrix are useful in data compression as it compress the size of a matrix while preserving sufficient information. More details can be found in this link https://www.datacamp.com/tutorial/mastering-low-rank-adaptation-lora-enhancing-large-language-models-for-efficient-adaptation. In short, we can use LoRA to train only a small subset of parameters or low-rank matrices associated with the layers we specify. We can select the layers that we want to decompose into low-rank matrix called LoRA adapter and train the adapters and add to the existing model during prediction.

In [ ]:
!pip3 install -i https://pypi.org/simple/ bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U datasets
!pip3 install -q -U transformers
!pip3 install accelerate

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import os

In [ ]:
model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
    

### Test the pretrained based model on sample questions

In [ ]:
text = "Human: Write a python program to remove duplicate elements from a list. Assistant:"
device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: Write a python program to remove duplicate elements from a list. Assistant: Use the built-in 'collections.Counter' class.

```python
# Create a list of duplicate elements
duplicate_list = [1, 2, 3, 4, 5, 1, 2, 3, 4, 5]

# Create a Counter object from the list
counter = collections.Counter(duplicate_list)

# Print the unique elements from the list
print(list(counter.elements()))
```

**Output:**

```
[1, 2, 3, 4, 5]
```

**Explanation:**

1


In [ ]:
text = "Human: Write a function to check if a given number is even or odd. Assistant:"
device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: Write a function to check if a given number is even or odd. Assistant: Sure, here's a function to check if a given number is even or odd:

```python
def is_even(number):
  """
  Checks if a given number is even.

  Args:
    number: The number to check.

  Returns:
    True if the number is even, False otherwise.
  """

  return number % 2 == 0


# Example usage
print(is_even(4))  # Output: True
print(is_even(5))  # Output: False
```

**Explanation:**

* The `is_


In [ ]:
text = "Human: What does if name == “main”: do? Assistant:"
device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: What does if name == “main”: do? Assistant: The code you provided is asking the question "What does if name == “main” do?".

Sure, here's a breakdown of the code you provided:

```python
if name == "main":
    # code to be executed if name is "main"
```

**Explanation:**

* `if` statement checks if the value of `name` is equal to the string "main".
* `name == "main"` is a condition that compares the value of `name` with the string "main".
* If the condition is true, the code inside the `if` block will be executed


In [ ]:
text = "Human: How do I merge two dictionaries in a single expression (taking union of dictionaries)? Assistant:"
device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: How do I merge two dictionaries in a single expression (taking union of dictionaries)? Assistant: Sure, here's how you can merge two dictionaries in a single expression (taking union of dictionaries):

```python
dict1 = {"key1": "value1", "key2": "value2"}
dict2 = {"key3": "value3", "key4": "value4"}

merged_dict = dict1.copy()
merged_dict.update(dict2)

print(merged_dict)
```

**Output:**

```
{"key1": "value1", "key2": "value2", "key3": "value3", "key4":


### Finetuning using LORA

In [ ]:
##we can see the layers of the model and select which layers we want to apply adapter weights.
##1_proj and v_proj are the attention layers.
##r -> the rank of the low rank matrices learned while finetuning. Larger r means a large number of parameters to be trained.
from peft import LoraConfig
lora_config = LoraConfig(
    r=8, #the rank of the low rank matrix
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], ##all the linear layers
    task_type="CAUSAL_LM",
)

In [ ]:
from datasets import load_dataset

data = load_dataset("KonradSzafer/stackoverflow_python_preprocessed")

#data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
#data.train_test_split(test_size=0.2)
#data = data["train"].train_test_split(test_size=0.2)
data = data.map(lambda samples: tokenizer(samples["question"]), batched=True)
print(data)

Map:   0%|          | 0/3296 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'answer', 'question', 'input_ids', 'attention_mask'],
        num_rows: 3296
    })
})


In [ ]:
import transformers
from trl import SFTTrainer

def formatting_func(example):
    text = f"Question: {example['question'][0]}\nanswer: {example['answer'][0]}"
    return [text]

trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=20,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3296 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
1,3.427800
2,3.427800
3,3.295200
4,3.017700
5,2.754700
6,2.524400
7,2.331600
8,2.173200
9,2.040600
10,1.923600


TrainOutput(global_step=20, training_loss=2.0820329904556276, metrics={'train_runtime': 43.7288, 'train_samples_per_second': 1.829, 'train_steps_per_second': 0.457, 'total_flos': 309030657638400.0, 'train_loss': 2.0820329904556276, 'epoch': 20.0})

### Test the finetuned model on example questions

In [ ]:
text = "Human: Write a python program to remove duplicate elements from a list. Assistant:"
device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: Write a python program to remove duplicate elements from a list. Assistant: I'd be happy to help!

```python
def remove_duplicates(nums):
    """
    Removes duplicate elements from a list.

    Args:
        nums (list): The list of elements to remove duplicates from.

    Returns:
        list: The list of elements with duplicate elements removed.
    """

    # Create a set of unique elements from the list.
    unique_elements = set(nums)

    # Return the list of unique elements.
    return list(unique_elements)


# Example usage.
nums = [1,2,3,4


In [ ]:
text = "Human: Write a function to check if a given number is even or odd. Assistant:"
device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: Write a function to check if a given number is even or odd. Assistant: Sure. Here's a function to check if a number is even or odd:

```python
def is_even(number):
  """
  Checks if a number is even.

  Args:
    number: The number to check.

  Returns:
    True if the number is even, False otherwise.
  """

  return number % 2 == 0


# Example usage
print(is_even(12))  # Output: True
print(is_even(11))  # Output: False
```

**Explanation:**

* The `is_


In [ ]:
text = "Human: What does if name == “main”: do? Assistant:"
device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: What does if name == “main”: do? Assistant: If the name of the current object is "main", this block of code will be executed.
Sure, here's an example:

```python
def my_function():
    if name == "main":
        print("I am in the main module!")

# Call the function
my_function()
```

Output:

```
I am in the main module!
```

So, in this example, the `my_function` is only executed if the current object's name is "main".


In [ ]:
text = "Human: How do I merge two dictionaries in a single expression (taking union of dictionaries)? Assistant:"
device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: How do I merge two dictionaries in a single expression (taking union of dictionaries)? Assistant: Sure, here's how you can merge two dictionaries in a single expression (taking union of dictionaries):

```python
dict1 = {'key1': 'value1', 'key2': 'value2'}
dict2 = {'key3': 'value3', 'key4': 'value4'}

merged_dict = dict1.copy()
merged_dict.update(dict2)

print(merged_dict)
```

Output:

```python
{'key1': 'value1', 'key2': 'value2', 'key3': 'value3', 'key4':


We can see that some answers are better after  finetuning the model on the specific python Q/A dataset. Though not perfect, but answer to the question "What does if name == “main”: do?" was much better after finetuning. Also, the answer to the question "Write a python program to remove duplicate elements from a list" was wrong when we used the pretrained base model. However, the finetuned model provided accurate answer to the question. We can play with the hyper-parameters while finetuning to get better performance. We need some way to assess the performance of the LLM.

Ref:
https://huggingface.co/blog/gemma-peft

https://www.datacamp.com/tutorial/mastering-low-rank-adaptation-lora-enhancing-large-language-models-for-efficient-adaptation